In [1]:
import findspark
findspark.init()


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NormalProcessor").getOrCreate()
spark.version


22/02/21 17:18:40 WARN Utils: Your hostname, sarbajit-laptop resolves to a loopback address: 127.0.1.1; using 192.168.0.130 instead (on interface wlp2s0)
22/02/21 17:18:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/21 17:18:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'3.1.2'

In [10]:
import sys
df = spark.read.format('csv').options(delimiter=',', header=True).load('/home/sarbajit/Documents/Programs/spark/Data/imp/Book1.csv')
df.show()
df = df.dropna(how="any",subset=['Exp'])
df.show()

# d = [
#     ('','123'),
#     ('23','234')
# ]

# df = spark.createDataFrame(d, schema=['a','b'])
# df.show()
# df = df.replace("",None)
# df.show()


# sys.exit()

+--------+---+---+
|    Name|Age|Exp|
+--------+---+---+
|   Ajith| 22|  2|
|Jyothish| 21|  3|
|  Anandu| 20|  5|
|    null| 18|  1|
+--------+---+---+

+--------+---+---+
|    Name|Age|Exp|
+--------+---+---+
|   Ajith| 22|  2|
|Jyothish| 21|  3|
|  Anandu| 20|  5|
|    null| 18|  1|
+--------+---+---+



In [ ]:
emp = spark.createDataFrame([],schema=['Name','Age','Exp'])

# emp=emp.union(df)
# emp.show()

In [ ]:
schema = {
    'fname' : {
        'datatype':'String',
        'trim' : 'Y'
    },

    'lname' : {
        'datatype':'String',
        'trim' : 'Y'
    },

    'id' : {
        'datatype':'Decimal',
        'trim' : 'Y',
        'scale' : 2,
        'precision' : 10
    }
}


In [ ]:
import pyspark.sql.types
def schemaParser():
    global schema
    needTypeCast=dict()
    allFields = []
    demoSchema = StructType()
    fieldSchema = StructType()
    trimFlag = []
    for key in schema.keys():
        allFields.append(key.title())
        if schema[key]['datatype']=='Decimal':
            fieldType = getattr(pyspark.sql.types, (schema[key]['datatype'] + 'Type'))(int(schema[key]['precision']), int(schema[key]['scale']))
        else:
            fieldType = getattr(pyspark.sql.types, (schema[key]['datatype'] + 'Type'))()
        fieldSchema.add(key.title(), fieldType)
        demoSchema.add(key.title(), StringType())
        if schema[key]['datatype']!='String':
            needTypeCast[key.title()]=fieldType

        if (schema[key]['trim']=='Y'):
            trimFlag.append(key.title())

    return fieldSchema,demoSchema ,trimFlag, needTypeCast

In [ ]:
import os
file_path = "../Data/imp/test.csv"

fieldSchema, allField, trimFlag, needTypeCast = schemaParser()
print(needTypeCast)

In [ ]:
from pyspark.sql.functions import udf
parse = udf(lambda record : go(record, ','), StringType())

In [ ]:
# read as single col
df = spark.read.format('text').option('delimiter','').load(file_path)
df.show()

In [ ]:
df = df.withColumn('value', parse(col('value')))
df.show()

In [ ]:
df = spark.read.format('csv').option('delimiter',',').schema(allField).load(file_path)
df.show()

In [ ]:
for column in trimFlag:
    df = df.withColumn(column, trim(col(column)))
df.show()

In [ ]:
for column in needTypeCast.keys():
    df = df.withColumn(column, col(column).cast(needTypeCast[column]))
df.show()

In [ ]:
sc.uiWebUrl

In [ ]:
spark.stop()

In [ ]:
import csv

import os
file_path = "../Data/imp/test.csv"

with open(file_path) as csv_file:
    rd = csv.reader(csv_file, delimiter=", ")
    for row in rd:
        print(",".join(row))

In [ ]:
def go(record, delimiter):
    record = record+','
    quoteIndexes = [ i for i, val in enumerate(record) if val=='"']
    if (len(quoteIndexes)&1):
        print("\" are present in data")
    
    print(quoteIndexes)
    i = 0
    j = len(record)
    print("j",j)
    cols = []
    # rec = record
    colIndexes= []
    count=0
    while (i<j):
        # print("org record->", record)
        # print("record    ->", rec)
        start =i
        # print("strt:", start)
        if record[i]==' ':
            i+=1
            continue

        if i in quoteIndexes:
            quoteIndexes.pop(0)
            end = quoteIndexes.pop(0) + 1
        else:
            # print(record[start:j+1])
            end = record[start:j+1].index(delimiter) + start
            if '"' in record[start:end]:
                quoteIndexes.pop(0)
                quoteIndexes.pop(0)


        # print((start, end)) 
        # colIndexes.append((start,end))
        cols.append(record[start:end])
        # print("value of i", i)
        i=end+1
        # count+=1
    
    # print(colIndexes)
    return delimiter.join(cols)



In [ ]:

def go(record, delimiter):
    record = record+delimiter
    # quoteIndexes = [ i for i, val in enumerate(record) if val=='"']
    quoteCount = 0
    quoteEndIndx = -1
    colEndIndexes=[]
    rec_len = len(record)
    for i, val in enumerate(record):
        if val=='"':
            if (quoteCount&1 and i<rec_len-1 and record[i+1]==delimiter) or (not quoteCount&1):
                quoteCount+=1
             
        if val==delimiter and not quoteCount&1:
            colEndIndexes.append(i)
    # print(colEndIndexes)
    start = 0
    indexLists = []
    for i in colEndIndexes:
        indexLists.append((start,i))
        start=i+1


    
    return delimiter.join([record[tup[0]:tup[1]].strip() for tup in indexLists])
    # return colEndIndexes



In [ ]:
data_sample = ['123, ABCDES,,, "Sarbajit "abd"s", Nandy", ABC, " 221B, Baker street"', 
               '123, "Sarbajit, Nandy", ABC,"221B, Baker street"', 
               '123,"Sarbajit, Nandy",ABC, "221B, Baker street"']
[go(data,",") for data in data_sample]
# 123, ABCDES,,,="SarbajitNandy", ABC, " 221B, Baker street"

In [ ]:
from pyspark.sql.functions import add_months, col, date_format, from_unixtime,unix_timestamp

date = '20210722'
s = [(date,)]
print(s)
df= spark.createDataFrame(s, schema=['value'])
df.show()
dfO = df.withColumn('value',date_format(add_months(from_unixtime(unix_timestamp(col('value'),'yyyyMMdd')),-12),'yyyyMMdd'))
dfO.show()

In [38]:
df = spark.createDataFrame([['12','20210506'],['11', None,], ['12','20201206'], ['14','20210131']], schema=['client','date'])
df.show()

+------+--------+
|client|    date|
+------+--------+
|    12|20210506|
|    11|    null|
|    12|20201206|
|    14|20210131|
+------+--------+



In [39]:
from pyspark.sql.functions import unix_timestamp,date_format,from_unixtime, col, to_date
df.createOrReplaceTempView("DF")

# spark.sql("selct date_format(from_unixtime(unix_timestamp(col('date'),'yyyyMMdd')),'yyyy-MM-dd') from DF").show()
# df = df.withColumn('newDate',date_format(from_unixtime(unix_timestamp(col('date'),'yyyyMMdd')),'yyyy-MM-dd'))
df = df.withColumn('newDate',to_date(col('date'),'yyyyMMdd'))

df.show()

+------+--------+----------+
|client|    date|   newDate|
+------+--------+----------+
|    12|20210506|2021-05-06|
|    11|    null|      null|
|    12|20201206|2020-12-06|
|    14|20210131|2021-01-31|
+------+--------+----------+



In [42]:
df.createOrReplaceTempView("DF")
spark.sql("select client,max(newDate) from DF group by client").show()

+------+------------+
|client|max(newDate)|
+------+------------+
|    11|        null|
|    12|  2021-05-06|
|    14|  2021-01-31|
+------+------------+



In [14]:
from pyspark.sql.types import StructType, StructField, StringType,DateType,TimestampType, IntegerType

In [15]:
schema = StructType()
schema.add('name', StringType(), True)
schema.add('name2', StringType(), True)
schema.add('name3', StringType(), True)

StructType(List(StructField(name,StringType,true),StructField(name2,StringType,true),StructField(name3,StringType,true)))

In [14]:
df = spark.read.format("csv")\
    .option("header",True)\
    .option('sep',",")\
    .option("multiLine", True)\
    .load('/home/sarbajit/Documents/Programs/spark/Data/test.csv')
df.show()
df.count()

+--------+----------+-----+
|    Name|       add|LName|
+--------+----------+-----+
|sarbajit|123,
Baker|nandy|
|sarbajit|       234|nandy|
|     sha|        34|  eno|
+--------+----------+-----+



3

In [15]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
def go(value):
    return str(value).replace("\n","")

feed = udf(go, StringType())

In [16]:


for each in df.columns:
    df = df.withColumn(each, feed(col(each)))

df.show()

+--------+---------+-----+
|    Name|      add|LName|
+--------+---------+-----+
|sarbajit|123,Baker|nandy|
|sarbajit|      234|nandy|
|     sha|       34|  eno|
+--------+---------+-----+



In [18]:
df.write.format("csv").option("path","df.csv").option("delimiter", ',').option("escape", "\n").save()

In [18]:
df.createOrReplaceTempView("DF")

spark.sql("select name2,name3  from DF").show()

+-----------------+-----+
|            name2|name3|
+-----------------+-----+
|asdasdasd,
werere|Nandy|
+-----------------+-----+



In [28]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- date: date (nullable = true)



In [3]:
rdd1 = spark.read.format('text').load("/home/sarbajit/Documents/Programs/spark/Data/json.json")
rdd1.show()

+--------------------+
|               value|
+--------------------+
|{"name":"sarbajit...|
|{"name":"Shalmoli...|
|{"name":"Supriya"...|
|{"name":"Shuvanka...|
+--------------------+



In [42]:
from pyspark.sql.functions import udf,col,lit, from_json
from pyspark.sql.types import StructType,StringType
import json

# JsonUdf = udf(lambda value : json.loads(value), StructType())
# for each in ['name','dept','c']:
#     rdd1 = rdd1.withColumn(each, lit(None).cast(StringType()))
# rdd1.show()
schema = StructType()
schema.add('c',StringType())
schema.add('name',StringType())
schema.add('dept',StringType())
rdd1.withColumn('value', from_json(col('value'), schema)).select(col('value.*')).show()

+----+---------+----+
|   c|     name|dept|
+----+---------+----+
|true| sarbajit| CSE|
|null| Shalmoli| CSE|
|null|  Supriya|  EE|
|null|Shuvankar|  IT|
+----+---------+----+



In [14]:
from pyspark.sql.functions import countDistinct
df = spark.read.format('json').load("/home/sarbajit/Documents/Programs/spark/Data/json.dat")
df.show()

cdf = df.select(countDistinct('name').alias("value"))

+-----+----+---------+
|    c|dept|     name|
+-----+----+---------+
| true| CSE| sarbajit|
| null| CSE| Shalmoli|
| null|  EE| sarbajit|
|false|  IT|Shuvankar|
+-----+----+---------+



In [13]:
cdf.take(1)[0]['value']

3

In [3]:
import json 
from pyspark.sql import Row
from pyspark.sql.types import *
rdd2 = df.rdd.map(lambda row : json.loads(row.value))
rdd2.collect()
rdd2 = rdd2.map(lambda value : Row(**value))
rdd2.collect()
# l= rdd2.collect()[0]
# print(l.keys())
schema = StructType()
schema.add('c',StringType(), True)
schema.add('name',StringType(), True)
schema.add('dept',StringType(), True)
df = spark.createDataFrame(rdd2, schema=schema)
df.show()
# rdd2.toDF().show()
# rdd2.saveAsTextFile("test.txt")

NameError: name 'df' is not defined

In [14]:
a = {'name': 'Alic', 'dept': 'cse'}
s = Row(**a)
s.asDict()

{'name': 'Alic', 'dept': 'cse'}

In [3]:
df=spark.createDataFrame([['04/12/2021 00:00:00'],['04/11/2021 12:00:00']], schema=['value'])
df.show()

In [13]:
from pyspark.sql.functions import to_date, substring,date_format

df.createOrReplaceTempView("DF")

spark.sql("select date_format(to_date(from_unixtime(unix_timestamp(substring(value,1,10),'MM/dd/yyyy'))),'yyyyMMdd') as DATE from DF").show()

+--------+
|    DATE|
+--------+
|20210412|
|20210411|
+--------+



In [38]:
# UDF
from pyspark.sql.types import StructType,StringType,IntegerType,StructField
schema = StructType([
    StructField('Name', StringType()),
    StructField('Age', StringType()),
    StructField('Exp', StringType())
])
df = spark.read.format('csv').options(delimiter=',', header=True).load('/home/sarbajit/Documents/Programs/spark/Data/imp/Book1.csv', schema=schema)
df.show()
df.printSchema()

+--------+---+---+
|    Name|Age|Exp|
+--------+---+---+
|   Ajith| 22|  2|
|Jyothish| 21|  3|
|  Anandu| 20|  5|
|        | 18|  1|
+--------+---+---+

root
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Exp: string (nullable = true)



In [17]:
from pyspark.sql.functions import udf, array , when, col, lit
from pyspark.sql.types import StringType

# df.select (array("Name","Age").alias('ARR')).show()
def contructNames(names) : 
    if not all([ type(each)==str for each in names]):
        raise ValueError("List contains non-string element")
    else:
        return " ".join([name.strip() for name in names ])

ConstructNameUDF = udf(contructNames, StringType())

arg = array("Name","Age","Exp")
# df.select(ConstructNameUDF(arg)).show()
df=df.withColumn("new", ConstructNameUDF(arg))

arg = array("Name","Age","Exp", lit('PASS'))
df =df.withColumn("new2", when((col('Name').isNull()) | (col('Name')==' '), col('new')).otherwise(ConstructNameUDF(arg)))
df.show()

AnalysisException: cannot resolve '`Age`' given input columns: [Department, Manager, Name, Salary];;
'Project [Name#61, Department#62, Manager#63, Salary#64, contructNames(array(Name#61, 'Age, 'Exp)) AS new#365]
+- Relation[Name#61,Department#62,Manager#63,Salary#64] csv


In [46]:
df2 = spark.createDataFrame([[1,10],[2,20],[3,30],[4,40],[5,50]], schema=['Exp','Age'])
df2.show()

+---+---+
|Exp|Age|
+---+---+
|  1| 10|
|  2| 20|
|  3| 30|
|  4| 40|
|  5| 50|
+---+---+



In [5]:
df = spark.read.format('csv').option('delimiter',',').option('header',True).load('/home/sarbajit/Documents/Programs/spark/Data/imp/emp.csv')
df.show()

+-------------+----------+-----------+------+
|         Name|Department|    Manager|Salary|
+-------------+----------+-----------+------+
|   Robin Hood|      null|       null|   200|
|Arsene Wenger|       Bar| Friar Tuck|    50|
|   Friar Tuck|       Foo| Robin Hood|   100|
|  Little John|       Foo| Robin Hood|   100|
|Sam Allardyce|      null|       null|   250|
|Dimi Berbatov|       Foo|Little John|    50|
+-------------+----------+-----------+------+



In [20]:
dfM = df.select("Name","Salary")
dfM.show()
cond = "ManagerSalary"
df.alias('A').join(dfM.alias('B'),how='left',on=[col("A.Manager")==col("B.Name")]).select("A.*", col('B.Salary').alias(cond)).show()

+-------------+------+
|         Name|Salary|
+-------------+------+
|   Robin Hood|   200|
|Arsene Wenger|    50|
|   Friar Tuck|   100|
|  Little John|   100|
|Sam Allardyce|   250|
|Dimi Berbatov|    50|
+-------------+------+

+-------------+----------+-----------+------+-------------+
|         Name|Department|    Manager|Salary|ManagerSalary|
+-------------+----------+-----------+------+-------------+
|   Robin Hood|      null|       null|   200|         null|
|Arsene Wenger|       Bar| Friar Tuck|    50|          100|
|   Friar Tuck|       Foo| Robin Hood|   100|          200|
|  Little John|       Foo| Robin Hood|   100|          200|
|Sam Allardyce|      null|       null|   250|         null|
|Dimi Berbatov|       Foo|Little John|    50|          100|
+-------------+----------+-----------+------+-------------+



In [20]:
from pyspark.sql.functions import array_contains
s = [
    ['id1', ['Open','Close','Unknown']],
    ['id2', ['Close','Unknown']],
    ['id3', None],
    ['id4', ['Unknown']]
]

s2 = [
    ['id1', 'Open'],
    ['id2', 'Unknown'],
    ['id3', 'Open'],
    ['id4', 'Open'],
    ['id4', 'Unknown'],
    ['id5', 'Open'],
    ['id6', None]
]

df = spark.createDataFrame(s, schema=['mp_id', 'actions'])
df2 = spark.createDataFrame(s2, schema=['mp_id', 'action'])
df.show()
df2.show()

df.createOrReplaceTempView("DF")
df2.createOrReplaceTempView("DF2")

dsql  = "SELECT * from DF a inner join DF2 b on a.mp_id=b.mp_id and ( a.actions is null or array_contains(a.actions, b.action))"
spark.sql(dsql).show()

+-----+--------------------+
|mp_id|             actions|
+-----+--------------------+
|  id1|[Open, Close, Unk...|
|  id2|    [Close, Unknown]|
|  id3|                null|
|  id4|           [Unknown]|
+-----+--------------------+

+-----+-------+
|mp_id| action|
+-----+-------+
|  id1|   Open|
|  id2|Unknown|
|  id3|   Open|
|  id4|   Open|
|  id4|Unknown|
|  id5|   Open|
|  id6|   null|
+-----+-------+



+-----+--------------------+-----+-------+
|mp_id|             actions|mp_id| action|
+-----+--------------------+-----+-------+
|  id3|                null|  id3|   Open|
|  id1|[Open, Close, Unk...|  id1|   Open|
|  id2|    [Close, Unknown]|  id2|Unknown|
|  id4|           [Unknown]|  id4|Unknown|
+-----+--------------------+-----+-------+



In [11]:
sc = SparkContext
rdd1 = spark.createDataFrame([
        (1,"Sarbajit"),
        (2,"Shalmoli"),
        (3,"Supriya"),
        (4,"Shuvo")
    ], schema=['id','name']).rdd

rdd1.collect()
rdd2 = spark.createDataFrame([
        (1,"CSE"),
        (2,"IT"),
        (3,"CSE"),
        (4,None)
    ], schema=['id','Dept']).rdd

rdd2.collect()




[Row(id=1, name='Sarbajit'),
 Row(id=2, name='Shalmoli'),
 Row(id=3, name='Supriya'),
 Row(id=4, name='Shuvo')]

In [11]:
from pyspark.sql.functions import array_contains, col
s = [
    ["c1",1],
    ["c2",1],
    ["c3",1],
    ["c1",0],
    ["c4",0],
    ["c1",1],
    ["c3",1]
]

df = spark.createDataFrame(s, schema=['client','ind'])
# df.show()
# df.createOrReplaceTempView("MAIN")

# sql = "select client,\
#      case when min(ind)=0 then 0 else 1 end Indicator\
#      from MAIN group by client"
# spark.sql(sql).show()

df.groupBy("client").min("ind").alias("indicator").show()

+------+--------+
|client|min(ind)|
+------+--------+
|    c1|       0|
|    c4|       0|
|    c3|       1|
|    c2|       1|
+------+--------+



In [4]:
data = (
    (1,'sarbajit',19),
    (2,'shalmoli',20),
    (3,'supriya',21),
    (4,'shuvankar',32),
)

spark.createDataFrame(data,schema=['id','name','roll']).show()

+---+---------+----+
| id|     name|roll|
+---+---------+----+
|  1| sarbajit|  19|
|  2| shalmoli|  20|
|  3|  supriya|  21|
|  4|shuvankar|  32|
+---+---------+----+



In [14]:
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType
d = [
     [1,'sarbajit'],
     [2,'Shal'],
     [3,'Sup'],
     [4,'Shu']
]
df = spark.createDataFrame(d, schema=['id','name'])
df.createOrReplaceTempView("DF")
df1 = spark.createDataFrame(d, schema=['id','name'])
df1.createOrReplaceTempView("DF1")
# spark.sql('select max(X.id) from DF X inner join DF Y on X.id<Y.id and Y.id=3').show()
sql = "SELECT CASE WHEN A.id in (3,4) then (select max(X.id) from DF X, DF Y where X.id<Y.id and Y.id=A.id) ELSE A.id END as NEW_ID from DF A"
spark.sql(sql).show()
# def D3(id):
#     global spark
#     print(id)
#     sql = f"select max(id) id from DF where id<{id}"
#     df= spark.sql(sql)
#     # df.show()
#     print(df.collect()[0]['id'])
#     return 12

# FUNC = udf(D3, IntegerType())
# df = df.withColumn("expected_batch_date", FUNC(col('id')))
# df.show()
# print(D3(3))

+------+
|NEW_ID|
+------+
|     1|
|     2|
|     2|
|     3|
+------+



In [11]:
spark.stop()